## Credit Risk PoC on Lamda by Kaushal and Amar

### Part (I) Simulation of Development Environment

In [ ]:
# Save Model Using Pickle
import pandas as pd
import numpy as np
import boto3
from sagemaker import get_execution_role
import sagemaker
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle

In [139]:
role = "arn:aws:iam::476054568469:role/SMTRN-us-east-1-SC-476054568469-pp-5kg5wagv7mzae"
kms_key="arn:aws:kms:us-east-1:476054568469:key/44d350fa-2751-42ce-8389-b9e98130da58"
extra_args={
    "ServerSideEncryption": "aws:kms",
    "SSEKMSKeyId": kms_key
}
subnets=['subnet-0be1d0f0d5b54e56d', 'subnet-052c7e710f9c47efb']
security_group_ids=['sg-0197fc87ee3393d12', 'sg-0a589e5446f15ae9f']

In [140]:
#location of data file 's3://kaushalcicd/lambdapoc2/diabetes.csv'
# and s3://mufg-data-sandbox-us-east-1-sagemaker/kaushal/diabetes.csv

In [141]:
bucket = "mufg-data-sandbox-us-east-1-sagemaker"
#bucket = "kaushalcicd"
sub_folder = "kaushal"
data_key = "diabetes.csv"
data_location = 's3://{}/{}/{}'.format(bucket, sub_folder, data_key)
data_location

's3://mufg-data-sandbox-us-east-1-sagemaker/kaushal/diabetes.csv'

In [142]:
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pd.read_csv(data_location, names=names, low_memory=False)
dataframe.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [143]:
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
test_size = 0.33
seed = 7
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)
X_test[1:5],Y_test[1:5]

(array([[  7.   , 181.   ,  84.   ,  21.   , 192.   ,  35.9  ,   0.586,
          51.   ],
        [ 13.   , 152.   ,  90.   ,  33.   ,  29.   ,  26.8  ,   0.731,
          43.   ],
        [  6.   ,  93.   ,  50.   ,  30.   ,  64.   ,  28.7  ,   0.356,
          23.   ],
        [  6.   , 125.   ,  78.   ,  31.   ,   0.   ,  27.6  ,   0.565,
          49.   ]]),
 array([1., 1., 0., 1.]))

In [144]:
# Fit the model on training set
model = LogisticRegression()
model.fit(X_train, Y_train)
# save the model to .pkl file in S3 bucket
#filename = 's3:\\kaushalcicd\lambdapoc2\kaushal_test.pkl'
# s3://mufg-data-sandbox-us-east-1-sagemaker/kaushal/kaushal_test.pkl
data_key = 'kaushal_test.pkl'
filename = 's3://{}/{}/{}'.format(bucket, sub_folder, data_key)
filename


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


's3://mufg-data-sandbox-us-east-1-sagemaker/kaushal/kaushal_test.pkl'

In [145]:
filename = 'kaushal1.pkl'
pickle.dump(model, open(filename, 'wb'))

In [146]:
!aws s3 cp kaushal1.pkl s3://mufg-data-sandbox-us-east-1-sagemaker/kaushal/kaushal_test.pkl --sse

upload: ./kaushal1.pkl to s3://mufg-data-sandbox-us-east-1-sagemaker/kaushal/kaushal_test.pkl


### Part (II) Simualation of Production Environment

In [147]:

# load the model from .pkl file i.e. the Model pipeline
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

0.7874015748031497


### Option I: Use loaded model from .pkl file to predict output from the samples

In [148]:
loaded_model.predict(X_test[1:4])

array([1., 1., 0.])

In [149]:
df=pd.DataFrame(X_test[1:4],columns = names[0:8])
df

,preg,plas,pres,skin,test,mass,pedi,age
0,7.0,181.0,84.0,21.0,192.0,35.9,0.586,51.0
1,13.0,152.0,90.0,33.0,29.0,26.8,0.731,43.0
2,6.0,93.0,50.0,30.0,64.0,28.7,0.356,23.0


### Option II: Create a JSON file to be picked up from a S3 bucket to perdict the model using .pkl file

In [150]:
input_json = df.to_json()
print(input_json)

{"preg":{"0":7.0,"1":13.0,"2":6.0},"plas":{"0":181.0,"1":152.0,"2":93.0},"pres":{"0":84.0,"1":90.0,"2":50.0},"skin":{"0":21.0,"1":33.0,"2":30.0},"test":{"0":192.0,"1":29.0,"2":64.0},"mass":{"0":35.9,"1":26.8,"2":28.7},"pedi":{"0":0.586,"1":0.731,"2":0.356},"age":{"0":51.0,"1":43.0,"2":23.0}}


In [151]:
## Insert code to put his file in S3 bucket
import json
#outfile = 's3:\\kaushalcicd\lambdapoc2\input_json'
with open('s3:\\kaushalcicd\lambdapoc2\input_json.json', 'w') as json_file:
    json.dump(input_json, json_file)
#json.dump(input_json,'s3://kaushalcicd/lambdapoc2/input_json')

In [152]:
data_key = "input_json.json"
inputfile_loc = 's3://{}/{}/{}'.format(bucket, sub_folder, data_key)
inputfile_loc

's3://mufg-data-sandbox-us-east-1-sagemaker/kaushal/input_json.json'

In [153]:
with open(inputfile_loc,'r') as datafile:
    data = json.load(datafile)
    df_json = pd.DataFrame(data)

FileNotFoundError: [Errno 2] No such file or directory: 's3://mufg-data-sandbox-us-east-1-sagemaker/kaushal/input_json.json'

In [106]:
X_test_json = pd.read_json(input_json)
X_test_json

,preg,plas,pres,skin,test,mass,pedi,age
0,7,181,84,21,192,35.9,0.586,51
1,13,152,90,33,29,26.8,0.731,43
2,6,93,50,30,64,28.7,0.356,23


In [107]:
#X_test_json = df.values[0:3,0:8]
loaded_model.predict(X_test_json.values)

array([1., 1., 0.])